# Data Reconciliation Economizer, Single Unit Example

This notebook demonstrates data reconciliation with a single unit model, an economizer.  Data for this example was generated by adding noise to supercritical power plant simulations.

### Why reconcile data?

Data reconciliation uses mass and energy balances along with redundant measurements to improve data quality by:

1. reducing measurement error,
2. ensuring measurements satisfy mass and energy balances, and
3. filling in unmeasured quantities.

Data reconciliation is used to refine process data before parameter estimation.

## 1.  Read Plant Data

The first step is to read in process data.  In this case, data was simulated by adding measurement error to supercritical steam cycle simulation results.  IDAES includes functions to read process data, convert units to match the models, and map data to the model.

In [ ]:
# IDAES module with functions to read, analyze and visualize plant data
import idaes.dmf.model_data as da

Process data is contained in two csv files, a data file and a metadata file.  The data file where the first column is row indexes and the first row is process measurement tags.  The index column has an entry for each data row, and is often a timestamp.  The metadata file contains information about the tags including units of measurement, description, and model mapping information.

Once the process data is read in, the data is assigned to bins based on the value in a given column, in this case gross power. Dividing the data into bins allows rough estimation of measurement uncertainty.

In [ ]:
# Read in data and column metadata
df, df_meta = da.read_data("plant_data.csv", "plant_data_meta.csv")
# Add bin information where the data is sorted into 5 MW bins based on the "GROSS_POWER" column
bin_count = da.bin_data(df, bin_by="POWER_GROSS", bin_no="bin_no", bin_nom="bin_power", bin_size=5e6)
# Calculate the standard deviation by bin for each column
bin_stdev = da.bin_stdev(df, bin_no="bin_no")

It can be useful to visualize the measurement data and estimated uncertainty.  The following creates box and whisker plots for each tag based on the data bins.

In [ ]:
# Create a pdf book of plots that shows box and whisker plots for each column by bin
import os
if not os.path.isfile("boiler_plot_book.pdf"):
    da.data_plot_book(df, file="boiler_plot_book.pdf", bin_nom="bin_power", xlabel="gross power (W)", metadata=df_meta)
# There should now be a data_plot_book.pdf file in this directory

## 2.  Create Unit model

Now that we have the plant data, we need to create a unit model that we can use for data reconciliation.  Although we need a model that has just mass and energy balances and feasibility constraints for the data reconciliation problem, we start with the full boiler flowsheet model here.  Using the same model for data reconciliation, parameter estimation, validation, and simulation reduces the work required to move between steps in the workflow.

Once the full model is created, constraints that are not needed for data reconciliation can be deactivated.

### Case Study:
In a supercritical coal-fired power plant, the boiler sub-system consists of multiple heat exchangers transferring heat from the combustion side to the water/steam side. The boiler system for this work (Figure 1) is based on a 550 MWe supercritical power plant. Note that water wall and platen superheater are modeled as a heater, while the rest of the units correspond to the shell and tube heat exchangers.
Note that boiler flowsheet block diagram shows the available measurements from plant data. *for demonstration purposes "plant data" has been trained at different operating loads and with noise. 

In [ ]:
from IPython.display import Image
Image("boiler_flowsheet.png")

In [ ]:
# Import models
from idaes.core import FlowsheetBlock
from idaes.power_generation.properties.IdealProp_FlueGas import FlueGasParameterBlock
from idaes.core.util.model_statistics import degrees_of_freedom
from idaes.generic_models.properties import iapws95
from idaes.power_generation.unit_models.boiler_heat_exchanger import (
    BoilerHeatExchanger, 
    TubeArrangement, 
    DeltaTMethod
)
import pyomo.environ as pyo
import boiler_data_rec as flowsheet

In [ ]:
# Create boiler flowsheet
# boiler flowsheet is imported from idaes/power_plant/flowsheets/supercritical_power_plant/boiler_subflowsheet_build.py
# see documentation for more details
m = flowsheet.main_datarec()

# main_datarec methods imports the models, build the flowsheet, 
# links the unit models, and initializes the flowsheet

# -------------------------------------------------------- #
# The final solve corresponds to a simulation case, with the following inputs: 
# fixed inputs (relative to a ~650 MW case): 
# Econommizer (water): F, h, P
# Reheater Inlet (Cold Reheat steam): F, h, P
# coal flowrate and stoichiometric ratio
# dimensions of units (tube diameter, # tubes, hx area, etc.)

In [ ]:
import pytest
assert pyo.value(m.fs.coal_flow) == pytest.approx(50.15, rel=1e-4)
assert pyo.value(m.fs.PlSH.heat_duty[0]) == pytest.approx(88880834.87, rel= 1e-3)

## 3.  Simplify to Mass and Energy Balances

For data reconciliation, the model should be reduced to mass and energy balances and potentially limited performance constraints to keep the results feasible.

In [ ]:
# Deactivate constraints for heat transfer (for all heat exchanger units)
# for example to deactivate heat trasnfer calculation in the economizer
# m.fs.ECON.overall_heat_transfer_coefficient_eqn.deactivate()
# m.fs.ECON.rcond_wall_eqn.deactivate()
# m.fs.ECON.hconv_shell_total_eqn.deactivate()
# m.fs.ECON.hconv_shell_conv_eqn.deactivate()
# m.fs.ECON.LMTD.deactivate()

# The method deactivate_performance deactivates the equations for all 
# heat exchangers in this flowsheet (Economizer, Primary Superheater, 
#                                    Finishing Superheater, Reheater)
flowsheet.deactivate_performance(m)

# Now the model has 12 DoF (deltaP shell and tube, and U - overall heat transfer coefficient in 4 hx models)

In [ ]:
assert degrees_of_freedom(m) == pytest.approx(12, rel=1e-4)

## 4. Map Data to the Model

Although the model mapping can be added to the tag metadata file, here we just add the mapping information to the tag metadata after reading the data.

In [ ]:
# Water/Steam Measurements ----------------------------------------------- #
# Economizer inputs
df_meta["BFW_F"]["reference_string"] = "m.fs.ECON.side_1.properties_in[:].flow_mol"
df_meta["BFW_T"]["reference_string"] = "m.fs.ECON.side_1.properties_in[:].temperature"
df_meta["BFW_P"]["reference_string"] = "m.fs.ECON.side_1.properties_in[:].pressure"

# Economizer outputs
df_meta["ECON_OUT_T"]["reference_string"] = "m.fs.ECON.side_1.properties_out[:].temperature"
df_meta["ECON_OUT_P"]["reference_string"] = "m.fs.ECON.side_1.properties_out[:].pressure"

# Reheater inputs 
df_meta['RHT_COLD_T']["reference_string"] = "m.fs.RH.side_1.properties_in[:].temperature"
df_meta['RHT_COLD_P']["reference_string"] = "m.fs.RH.side_1.properties_in[:].pressure"
df_meta['RHT_COLD_F']["reference_string"] = "m.fs.RH.side_1.properties_in[:].flow_mol"

# Reheater outputs 
df_meta['RHT_HOT_T']["reference_string"] = "m.fs.RH.side_1.properties_out[:].temperature"
df_meta['RHT_HOT_P']["reference_string"] = "m.fs.RH.side_1.properties_out[:].pressure"

# primary SH outlet
df_meta['PlatenSH_IN_P']["reference_string"] = "m.fs.PrSH.side_1.properties_out[:].pressure"

# Platen SH outlet inlet to Finishing SH
df_meta['FSH_In_P']["reference_string"] = "m.fs.PrSH.side_1.properties_in[:].pressure"

# Finishing SH outlet ----------------------------------------------------- #
# Main Steam 
df_meta['MS_T']["reference_string"] = "m.fs.ATMP1.mixed_state[:].temperature"
df_meta['MS_P']["reference_string"] = "m.fs.ATMP1.mixed_state[:].pressure"
df_meta['MS_F']["reference_string"] = "m.fs.ATMP1.mixed_state[:].flow_mol"

# Flue Gas measurements
# Finishing inputs
df_meta['FG_2_FSH_Fm']["reference_string"] = "m.fs.FSH.side_2.properties_in[:].flow_mass"
df_meta['FG_2_FSH_T']["reference_string"] = "m.fs.FSH.side_2.properties_in[:].temperature"
df_meta['FG_2_FSH_P']["reference_string"] = "m.fs.FSH.side_2.properties_in[:].pressure"

# Reheater inputs
df_meta['FG_2_RH_T']["reference_string"] = "m.fs.RH.side_2.properties_in[:].temperature"
df_meta['FG_2_RH_P']["reference_string"] = "m.fs.RH.side_2.properties_in[:].pressure"

# Primary SH inputs
df_meta['FG_2_PrSH_Fm']["reference_string"] = "m.fs.PrSH.side_2.properties_in[:].flow_mass"
df_meta['FG_2_PrSH_T']["reference_string"] = "m.fs.PrSH.side_2.properties_in[:].temperature"
df_meta['FG_2_PrSH_P']["reference_string"] = "m.fs.PrSH.side_2.properties_in[:].pressure"

# Economizer inputs
df_meta["FG_2_ECON_P"]["reference_string"] = "m.fs.ECON.side_2.properties_in[:].pressure"

# Economizer outputs
df_meta["FG_2_AIRPH_T"]["reference_string"] = "m.fs.ECON.side_2.properties_out[:].temperature"
df_meta["FG_2_AIRPH_P"]["reference_string"] = "m.fs.ECON.side_2.properties_out[:].pressure"

# --------------------------------------------------------------------------------------- #
# Other inputs available in the data set (but not used as measurements in this study)
df_meta["ECON_OUT_F"]["reference_string"] = "m.fs.ECON.side_1.properties_out[:].flow_mol"
df_meta['RHT_HOT_F']["reference_string"] = "m.fs.RH.side_1.properties_out[:].flow_mol"
df_meta['FG_2_RH_Fm']["reference_string"] = "m.fs.RH.side_2.properties_in[:].flow_mass"
df_meta["FG_2_ECON_Fm"]["reference_string"] = "m.fs.ECON.side_2.properties_in[:].flow_mass"
df_meta["FG_2_ECON_T"]["reference_string"] = "m.fs.ECON.side_2.properties_in[:].temperature"
df_meta["FG_2_AIRPH_Fm"]["reference_string"] = "m.fs.ECON.side_2.properties_out[:].flow_mass"

df_meta['FG_RH_2_Mix_Fm']["reference_string"] = "m.fs.RH.side_2.properties_out[:].flow_mass"
df_meta['FG_RH_2_Mix_T']["reference_string"] = "m.fs.RH.side_2.properties_out[:].temperature"
df_meta['FG_RH_2_Mix_P']["reference_string"] = "m.fs.RH.side_2.properties_out[:].pressure"

df_meta['FG_PrSH_2_Mix_Fm']["reference_string"] = "m.fs.PrSH.side_2.properties_out[:].flow_mass"
df_meta['FG_PrSH_2_Mix_T']["reference_string"] = "m.fs.PrSH.side_2.properties_out[:].temperature"
df_meta['FG_PrSH_2_Mix_P']["reference_string"] = "m.fs.PrSH.side_2.properties_out[:].pressure"

df_meta['FG_2_ECON_Fm']["reference_string"] = "m.fs.ECON.side_2.properties_in[:].flow_mass"
df_meta['FG_2_ECON_T']["reference_string"] = "m.fs.ECON.side_2.properties_in[:].temperature"
df_meta['FG_2_ECON_P']["reference_string"] = "m.fs.ECON.side_2.properties_in[:].pressure"

df_meta['FG_2_AIRPH_Fm']["reference_string"] = "m.fs.ECON.side_2.properties_out[:].flow_mass"
df_meta['FG_2_AIRPH_T']["reference_string"] = "m.fs.ECON.side_2.properties_out[:].temperature"
df_meta['FG_2_AIRPH_P']["reference_string"] = "m.fs.ECON.side_2.properties_out[:].pressure"

In [ ]:
# Add the model references to the tag metadata based on the strings above.
da.upadate_metadata_model_references(m, df_meta)

In [ ]:
# Create a dictionary of data tags that we want to use for the data reconciliation problem.  
# The key is the tag and the value is a reference to a quantity in the model.  
data_tags = {k:v["reference"][0] for k, v in df_meta.items() if v["reference"] is not None}

In [ ]:
# Now for result output, the data reconciliation usually can give full stream information for a flowsheet
# including quantities that are unmeasured. To more easily use the results, it is good practice to map most of
# the data reconciliation results to flowsheet stream names.  
import idaes.core.util.tables as ta

stream_dict = ta.arcs_to_stream_dict(
    m, 
    additional={
        'MS': m.fs.ATMP1.mixed_state,
        'ATMP_In': m.fs.FSH.side_1.properties_out,
        'FSH_In': m.fs.FSH.side_1.properties_in,
        'PrSH_IN': m.fs.PrSH.side_1.properties_in,
        'RHT_COLD': m.fs.RH.side_1.properties_in,
        'RHT_HOT': m.fs.RH.side_1.properties_out,
        'PlatenSH_IN': m.fs.PlSH.control_volume.properties_in,
        'BFW': m.fs.ECON.side_1.properties_in,
        'ECON_OUT': m.fs.ECON.side_1.properties_out,
        'FG_2_FSH': m.fs.FSH.side_2.properties_in,
    },
    sort=True,
)

state_dict = ta.stream_states_dict(stream_dict, time_point=0)
recon_tags = ta.tag_state_quantities(
    blocks=state_dict, 
    attributes=(
        "flow_mass", 
        "flow_mol", 
        "enth_mol", 
        "temperature", 
        "pressure", 
        ("flow_component", "O2"),
        ("flow_component", "NO"),
        ("flow_component", "N2"),
        ("flow_component", "SO2"),
        ("flow_component", "CO2"),
        ("flow_component", "H2O"),
    ), 
    labels=("_Fm", "_F", "_h", "_T", "_P", "_F[O2]", "_F[NO]", "_F[N2]", "_F[SO2]", "_F[CO2]", "_F[H2O]"),
)
recon_tags["coal_flow"] = m.fs.coal_flow
recon_tags["stoich_ratio"] = m.fs.SR

## 5. View model flowsheet

Model results or other quantities can be added to a process flow diagram.  The PFD was drawn beforehand and the model results are added to tagged locations on the PFD.

In [ ]:
from idaes.core.util.misc import svg_tag  # utility to place numbers/text in an SVG

with open("boiler_scpc_PFD.svg", "r") as f:
    s = svg_tag(svg=f, tags={"subtitle":"Initialized Model"})
    s = svg_tag(svg=s, tags=data_tags, outfile="boiler_scpc_init.svg")

In [ ]:
from IPython.display import SVG, display

display(SVG(s))

## 6.  Write Objective

Next we write the objective function and additional constraints for the data reconciliation problem.  The objective is

$$\min \sum_i \left(\frac{x_{\text{data}, i} - x_{\text{model}, i}}{\sigma_i} \right)^2$$

Where $i \in \{\text{Measured Quantities}\}$ and $\sigma_i$ is the standard deviation of measurement i estimated by binning the data.

In [ ]:
# Add model parameters to contain measured data.  These are mutable so we can set a specific 
# change to specific data points later.
m.data = pyo.Param(data_tags, mutable=True, doc="Process data for a specific point in time.")
m.data_stdev = pyo.Param(data_tags, mutable=True, doc="Process data standard deviation.")

In [ ]:
# The 'set_data' function below takes data from the model DataFrame and updates the
# model data parameters.
# fix tags dictionary
_fix_tags = ['BFW_P',
            'BFW_F',
            'RHT_COLD_P',
            'RHT_COLD_F',]
def set_data(m, df, data_tags, index=None, indexindex=None):
    if index is None:
        index = df.index[indexindex]
    m.bin_no = df.iloc[index]["bin_no"]
    for t in data_tags:
        m.data[t] = df.iloc[index][t]
        m.data_stdev[t] = bin_stdev[m.bin_no][t]
    for tag in _fix_tags:
        data_tags[tag].fix(pyo.value(m.data[tag]))

In [ ]:
# So we have something reasonable to start, set the data attached to the model to the first 
# data point.
set_data(m, df, data_tags, indexindex=0)

Add an expression for error divided by the standard deviation, and use it to write the data reconciliation objective function.

In [ ]:
m.tags_obj = ["ECON_OUT_T", "ECON_OUT_P",
              "RHT_HOT_T", "RHT_HOT_T",
              "MS_T", "MS_P", "FG_2_ECON_P",
              "FG_2_AIRPH_T", "FG_2_AIRPH_P"]

@m.Expression(data_tags)
def err(m, i):
    return (m.data[i] - data_tags[i])/m.data_stdev[i]

m.objective = pyo.Objective(expr=sum(m.err[t]**2 for t in m.tags_obj))

In [ ]:
assert pyo.value(m.err["BFW_F"]) == pytest.approx(0.0, rel=1e-4)
assert pyo.value(m.err["BFW_P"]) == pytest.approx(0.0, rel=1e-4)
assert pyo.value(m.err["BFW_T"]) == pytest.approx(2.3, rel=1e-1)

assert pyo.value(m.err["RHT_COLD_F"]) == pytest.approx(0.0, rel=1e-4)
assert pyo.value(m.err["RHT_COLD_P"]) == pytest.approx(0.0, rel=1e-4)
assert pyo.value(m.err["RHT_COLD_T"]) == pytest.approx(0.8, rel=1)

## 7. Solve Optimization

Now we need to solve the data reconciliation problem for every data point.  The important results are stored in two DataFrames ```df_result``` which contains results tagged based on model stream names to be used in the parameter estimation step and ```df_result_cmp``` which contains reconciled data based on the original measurement tags and can be used to compare the original measurements to the reconciled results.

In [ ]:
# Add constraints or fix variable bounds to ensure reasonable flowsheet performance (i.e. LMTD, negative pressure drop, etc.).
# In this case, before solving the data reconciliation problem, unfix coal flowarate, stoichiometric ratio, and split fraction
# split fraction of the flue gas splitter to the reheater and primary superheater (see figure 1)
# surrogate models have been trained using a rigorous boiler fire side model with fixed wall temperature 
# and variable fuel flowrate and stoichiometric ratio. Models are valid between 30 to 70 kg/s of coal and 1.0 to 2.5 SR. 
# This represents a simplified boiler fire side and water side integration.
m.fs.coal_flow.unfix()
m.fs.coal_flow.setub(70.0)
m.fs.coal_flow.setlb(30.0)
m.fs.SR.unfix() # stoichiometric ratio 
m.fs.Spl1.split_fraction[0,'outlet_1'].unfix()

# Finishing superheater inlet conditions are determined by the surrogate models
# Surrogate models are used for Flue Gas Exit Temperature, component molar flowrate,
# and heat duty to the platen superheater and water wall (see documentation for more information)
m.fs.FSH.side_2.properties_in[:].temperature.unfix()
m.fs.FSH.side_2.properties_in[0].temperature.setub(1800)
m.fs.FSH.side_2.properties_in[0].temperature.setlb(1200)

# remove bounds if required only
# strip_bounds = pyo.TransformationFactory('contrib.strip_var_bounds')
# strip_bounds.apply_to(m, reversible=True)

In [ ]:
solver = pyo.SolverFactory('ipopt')

In [ ]:
import pandas as pd
# add bin information to reconciliation results so it can be used in parameter estimation
df_result = pd.DataFrame(columns=list(recon_tags.keys())+["termination", "bin_no", "bin_power"], index=df.index)
df_result_cmp = pd.DataFrame(columns=list(data_tags.keys())+["termination"], index=df.index)

for i in df.index:
    set_data(m, df, data_tags, index=i)
    ECON_T = m.data['BFW_T'].value
    ECON_P = m.data['BFW_P'].value
    hin1 = iapws95.htpx(ECON_T, ECON_P)
    m.fs.ECON.side_1.properties_in[0].enth_mol.fix(hin1)
    RH_P = m.data['RHT_COLD_P'].value
    RH_T = m.data['RHT_COLD_T'].value
    hin2 = iapws95.htpx(float(RH_T), float(RH_P))
    m.fs.RH.side_1.properties_in[0].enth_mol.fix(hin2)
    
    res = solver.solve(m)
    tc = str(res.solver.termination_condition)
    df_result.iloc[i]["termination"] = tc
    df_result.iloc[i]["bin_no"] = df.iloc[i]["bin_no"]
    df_result.iloc[i]["bin_power"] = df.iloc[i]["bin_power"]
    df_result_cmp.iloc[i]["termination"] = tc
    for t in recon_tags:
        df_result.iloc[i][t] = pyo.value(recon_tags[t])
    for t in data_tags:
        df_result_cmp.iloc[i][t] = pyo.value(data_tags[t])
    # Show something so you can tell progress is happening
    print(f"{i} -- {tc}, objective: {pyo.value(m.objective)}")
    # activate if user wants to terminate loop earlier
    #if i == 5:
     #   break

In [ ]:
# Save the reconciled data to be used for parameter estimation
df_result.to_csv("boiler_plant_recon.csv")

In [ ]:
# Create a new plot book to compare the original data to the reconciled data.
da.data_rec_plot_book(
    file="boiler_data_rec_plot_book.pdf",
    df_data=df, 
    df_rec=df_result_cmp,
    bin_nom="bin_power", 
    xlabel="gross power (W)", 
    metadata=df_meta
)